### ITESM Instituto Tecnológico de Estudios Superiores de Monterrey
### Course:     MLOps Machine Learning Operations
#### Teacher:   Carlos Mejia
#### Student:   Francisco Javier Torres Zenón  A01688757

### References
* Dataset and baseline notebook copied from 
[Online Payments Fraud Detection Dataset | Kaggle](https://www.kaggle.com/datasets/rupakroy/online-payments-fraud-detection-dataset) 
* Dataset: 
https://www.kaggle.com/datasets/rupakroy/online-payments-fraud-detection-dataset/download?datasetVersionNumber=1
* Original Baseline notebook: https://www.kaggle.com/code/nehahatti/online-payments-fraud-detection-project/notebook

### About this notebook
This notebook was taken from [Kaggle](http://www.kaggle.com) and has been rewriten 
for better reading and understanding.

1. The task is to predict online payment fraud, given a number of features from online 
transfer/deposits transactions.
2. On Kaggle there were several notebooks related to this dataset
(Decision Tree, Logistic Regresion, KNN, Gradient Boosting Classifier).
3. As a Baseline I choose one of the most accurated and simpler one, a notebook using
 the Decision Tree algorithm.

#### 1. SETUP 
Imports and configuration files needed to run this notebook


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import os

import joblib
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


for dirname, _, filenames in os.walk("/kaggle/input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# import os # This is an unused import

#### 2. Data Load
Instructions to load the dataset.

Please unload the dataset from: 
* Dataset: https://www.kaggle.com/datasets/rupakroy/online-payments-fraud-detection-dataset/download?datasetVersionNumber=1

and unzip the file ```PS_20174392719_1491204439457_log.csv``` in the folder ```../files/```

In [4]:
INPUT_FILES_PATH = "./Refactor/mlops_project/mlops_project/data/"
INPUT_FILE_NAME = "PS_20174392719_1491204439457_log.csv"
INPUT_FILE = INPUT_FILES_PATH + INPUT_FILE_NAME
Original_python_DIRECTORY='/Users/francisco.torres/Documents/GitHub/MLOps_project/'

#Change location to the refactored directory
print(os.getcwd())
os.chdir(Original_python_DIRECTORY)
print(os.getcwd())

Transactions_df = pd.read_csv(INPUT_FILE)

c:\Users\francisco.torres\Documents\GitHub\MLOps_project\Refactor
c:\Users\francisco.torres\Documents\GitHub\MLOps_project


#### 3. Data Exploration
Data analysis and visualization

In [5]:
Transactions_df.head(10)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.0,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.0,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.0,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.0,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.0,0.00,0,0
5,1,PAYMENT,7817.71,C90045638,53860.00,46042.29,M573487274,0.0,0.00,0,0
6,1,PAYMENT,7107.77,C154988899,183195.00,176087.23,M408069119,0.0,0.00,0,0
7,1,PAYMENT,7861.64,C1912850431,176087.23,168225.59,M633326333,0.0,0.00,0,0
8,1,PAYMENT,4024.36,C1265012928,2671.00,0.00,M1176932104,0.0,0.00,0,0
9,1,DEBIT,5337.77,C712410124,41720.00,36382.23,C195600860,41898.0,40348.79,0,0


In [6]:
Transactions_df.shape

(6362620, 11)

In [7]:
Transactions_df.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,8.338831e+05,8.551137e+05,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.888243e+06,2.924049e+06,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087215e+05,1.073152e+05,1.442584e+05,9.430367e+05,1.111909e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


In [8]:
Transactions_df.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [9]:
Transactions_df.dtypes

step                int64
type               object
amount            float64
nameOrig           object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest           object
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
isFlaggedFraud      int64
dtype: object

In [10]:
Transactions_df.type.value_counts()

type
CASH_OUT    2237500
PAYMENT     2151495
CASH_IN     1399284
TRANSFER     532909
DEBIT         41432
Name: count, dtype: int64

In [11]:
type = Transactions_df["type"].value_counts()
type

type
CASH_OUT    2237500
PAYMENT     2151495
CASH_IN     1399284
TRANSFER     532909
DEBIT         41432
Name: count, dtype: int64

In [12]:
# print transaction type counts
transaction = type.index
print(transaction)
quantity = type.value_counts()
print(quantity)

Index(['CASH_OUT', 'PAYMENT', 'CASH_IN', 'TRANSFER', 'DEBIT'], dtype='object', name='type')
count
2237500    1
2151495    1
1399284    1
532909     1
41432      1
Name: count, dtype: int64


In [13]:
# show dataframe
Transactions_df.describe

<bound method NDFrame.describe of          step      type      amount     nameOrig  oldbalanceOrg  \
0           1   PAYMENT     9839.64  C1231006815      170136.00   
1           1   PAYMENT     1864.28  C1666544295       21249.00   
2           1  TRANSFER      181.00  C1305486145         181.00   
3           1  CASH_OUT      181.00   C840083671         181.00   
4           1   PAYMENT    11668.14  C2048537720       41554.00   
...       ...       ...         ...          ...            ...   
6362615   743  CASH_OUT   339682.13   C786484425      339682.13   
6362616   743  TRANSFER  6311409.28  C1529008245     6311409.28   
6362617   743  CASH_OUT  6311409.28  C1162922333     6311409.28   
6362618   743  TRANSFER   850002.52  C1685995037      850002.52   
6362619   743  CASH_OUT   850002.52  C1280323807      850002.52   

         newbalanceOrig     nameDest  oldbalanceDest  newbalanceDest  isFraud  \
0             160296.36  M1979787155            0.00            0.00        0   

#### 4. Data transformation
Data cleaning, data transformation.

In [14]:
# Transform categorical attribute 'type'
Transactions_df["type"] = Transactions_df["type"].map(
    {"CASH_OUT": 1, "PAYMENT": 2, "CASH_IN": 3, "TRANSFER": 4, "DEBIT": 5}
)

In [15]:
# Transactions_df['isFraud'] = Transactions_df['isFraud'].map({0:'No_Fraud', 1:'Fraud'})

# select the numerical attributes

Transactions_df = Transactions_df[
    [
        "type",
        "amount",
        "oldbalanceOrg",
        "newbalanceOrig",
        "oldbalanceDest",
        "newbalanceDest",
        "isFraud",
    ]
]

In [16]:
Transactions_df.head(10)

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,2,9839.64,170136.00,160296.36,0.0,0.00,0
1,2,1864.28,21249.00,19384.72,0.0,0.00,0
2,4,181.00,181.00,0.00,0.0,0.00,1
3,1,181.00,181.00,0.00,21182.0,0.00,1
4,2,11668.14,41554.00,29885.86,0.0,0.00,0
5,2,7817.71,53860.00,46042.29,0.0,0.00,0
6,2,7107.77,183195.00,176087.23,0.0,0.00,0
7,2,7861.64,176087.23,168225.59,0.0,0.00,0
8,2,4024.36,2671.00,0.00,0.0,0.00,0
9,5,5337.77,41720.00,36382.23,41898.0,40348.79,0


In [17]:
correlation_summary = Transactions_df.corr()
correlation_summary

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
type,1.000000,0.198987,0.260418,0.270669,0.066255,0.079111,0.016171
amount,0.198987,1.000000,-0.002762,-0.007861,0.294137,0.459304,0.076688
oldbalanceOrg,0.260418,-0.002762,1.000000,0.998803,0.066243,0.042029,0.010154
newbalanceOrig,0.270669,-0.007861,0.998803,1.000000,0.067812,0.041837,-0.008148
oldbalanceDest,0.066255,0.294137,0.066243,0.067812,1.000000,0.976569,-0.005885
newbalanceDest,0.079111,0.459304,0.042029,0.041837,0.976569,1.000000,0.000535
isFraud,0.016171,0.076688,0.010154,-0.008148,-0.005885,0.000535,1.000000


In [18]:
correlation_summary["isFraud"].sort_values(ascending=False)

isFraud           1.000000
amount            0.076688
type              0.016171
oldbalanceOrg     0.010154
newbalanceDest    0.000535
oldbalanceDest   -0.005885
newbalanceOrig   -0.008148
Name: isFraud, dtype: float64

#### 5. Model training
* Load sklearn Decision Tree Algorithm
* Split the dataset
* Train the model

In [19]:
numerical_features = np.array(
    Transactions_df[["type", "amount", "oldbalanceOrg", "newbalanceOrig"]]
)

In [20]:
var_objective = np.array(Transactions_df[["isFraud"]])

In [21]:
X_train, X_test, y_train, y_test = train_test_split(
    numerical_features, var_objective, test_size=0.2, random_state=3
)

In [22]:
DecisionTree_model = DecisionTreeClassifier()

In [23]:
DecisionTree_model.fit(X_train, y_train)

DecisionTreeClassifier()

In [24]:
DecisionTree_model.score(X_test, y_test)

0.9996785915236176

In [25]:
y_pred = DecisionTree_model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[1270690     180]
 [    229    1425]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1270870
           1       0.89      0.86      0.87      1654

    accuracy                           1.00   1272524
   macro avg       0.94      0.93      0.94   1272524
weighted avg       1.00      1.00      1.00   1272524



In [28]:
DecisionTree_model.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

### 6. Save Model

In [26]:
TRAINED_MODEL_DIR = "../trained_models/"
PIPELINE_NAME = "decision_tree"
PIPELINE_SAVE_FILE = f"{PIPELINE_NAME}_output.pkl"

# Save the model using joblib
MODEL_FILED = TRAINED_MODEL_DIR + PIPELINE_SAVE_FILE
joblib.dump(DecisionTree_model, MODEL_FILED)

FileNotFoundError: [Errno 2] No such file or directory: '../trained_models/decision_tree_output.pkl'

### 7. Load saved model and predict data

In [67]:
# Load saved model
DecisionTree_loaded = joblib.load(MODEL_FILED)

In [68]:
# NO_FRAUD test
test_data = np.array([[5, 7880, 7880, 0.0]])
DecisionTree_loaded.predict(test_data)[0]

0

In [69]:
# FRAUD test
test_data = np.array([[6, 80000, 80000, 0.0]])
DecisionTree_loaded.predict(test_data)[0]

1